In [83]:
import pandas as pd
import numpy as np
import time


df_train1 = pd.read_csv("train2.csv")
df_test1 = pd.read_csv("test2.csv")
del df_train1['Unnamed: 0']
del df_test1['Unnamed: 0']


import numpy as np
import time
df_train = df_train1[:]
df_test = df_test1[:]



import warnings
warnings.filterwarnings("ignore")
cols_to_use = ['name','desc']
len_feats = ['name_len','desc_len']
count_feats = ['name_count','desc_count']

for i in np.arange(2):
    df_train[len_feats[i]] = df_train[cols_to_use[i]].apply(str).apply(len)
    df_train[count_feats[i]] = df_train[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))
    
    
for i in np.arange(2):
    df_test[len_feats[i]] = df_test[cols_to_use[i]].apply(str).apply(len)
    df_test[count_feats[i]] = df_test[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))
    
    
    
#Flag 1 based on goal
thre = 700000
df_test_decided_1  = df_test[df_test.goal>=thre]
df_test_1  = df_test[df_test.goal<thre]
df_train_decided_1  = df_train[df_train.goal>=thre]
df_train_1  = df_train[df_train.goal<thre]

df_test_decided_1['final_status'] = 0

#Flag 2 based on disable_communication

df_test_decided_2  = df_test_1[df_test_1.disable_communication ==1 ]
df_test_2  = df_test_1[df_test_1.disable_communication !=1]
df_train_decided_2  = df_train_1[df_train_1.disable_communication ==1 ]
df_train_2  = df_train_1[df_train_1.disable_communication !=1]

df_test_decided_2['final_status'] = 0

for df in [df_test_decided_2,df_test_2,df_train_decided_2,df_train_2]:
    del df['disable_communication']
    

#Flag 2 based on   deadline-state_changed_at

df_test_decided_3  = df_test_2[df_test_2['deadline-state_changed_at'].values<0]
df_test_3  = df_test_2[df_test_2['deadline-state_changed_at'].values>=0]
df_train_decided_3  = df_train_2[df_train_2['deadline-state_changed_at'].values<0]
df_train_3  = df_train_2[df_train_2['deadline-state_changed_at'].values>=0]

df_test_decided_3['final_status'] = 0



In [93]:
from textblob import TextBlob
polarity = []
subjectivity = []
desc= []
for i in df_train_3['desc'].values:
    if str(i) == 'nan':
        i = 'none'
        b = TextBlob(i)
        x = b.sentiment
        polarity.append(x.polarity)
        subjectivity.append(x.subjectivity)
        desc.append(i)
    else:
        i = i.decode('utf8').encode('ascii', errors='ignore')
        b = TextBlob(i)
        x = b.sentiment
        polarity.append(x.polarity)
        subjectivity.append(x.subjectivity)
        desc.append(i)
df_train_3['polarity'] =polarity
df_train_3['subjectivity'] =subjectivity
df_train_3['desc'] = desc




from textblob import TextBlob
polarity = []
subjectivity = []
desc= []
for i in df_test_3['desc'].values:
    if str(i) == 'nan':
        i = 'none'
        b = TextBlob(i)
        x = b.sentiment
        polarity.append(x.polarity)
        subjectivity.append(x.subjectivity)
        desc.append(i)
    else:
        i = i.decode('utf8').encode('ascii', errors='ignore')
        i = i.replace('"','')
        b = TextBlob(i)
        x = b.sentiment
        polarity.append(x.polarity)
        subjectivity.append(x.subjectivity)
        desc.append(i)
df_test_3['polarity'] =polarity
df_test_3['subjectivity'] =subjectivity
df_test_3['desc'] = desc

In [126]:
df_test_3.columns

Index([u'project_id', u'name', u'desc', u'goal', u'keywords', u'country',
       u'currency', u'deadline', u'state_changed_at', u'created_at',
       u'launched_at', u'deadline-state_changed_at', u'launched_at-created_at',
       u'deadline-launched_at', u'deadline-created_at', u'keys_no', u'keys',
       u'keys_len', u'name_len', u'name_count', u'desc_len', u'desc_count',
       u'polarity', u'subjectivity'],
      dtype='object')

In [124]:
train = df_train_3
test = df_test_3
from sklearn.preprocessing import LabelEncoder

feat = ['country']
for x in feat:
    le = LabelEncoder()
    le.fit(list(train[x].values) + list(test[x].values))
    train[x] = le.transform(list(train[x]))
    test[x] = le.transform(list(test[x].values))

In [134]:
cols_to_use = ['name_len','desc_len','keys_len','name_count','desc_count','keys_no',
              'goal','country','deadline-state_changed_at','deadline-created_at']


target = train['final_status']

# data for modeling
k_train = train[cols_to_use]
k_test = test[cols_to_use]


In [135]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
nvb_pred = gnb.fit(k_train,target ).predict(k_test)

In [136]:
nBsub = pd.DataFrame({'project_id':test['project_id'],'final_status':nvb_pred})
nBsub = nBsub[['project_id','final_status']]

In [137]:
frames = [nBsub ,df_test_decided_1[['project_id','final_status']],df_test_decided_2[['project_id','final_status']],df_test_decided_3[['project_id','final_status']]]

In [138]:
nBsub = pd.concat(frames)

In [139]:
nBsub.to_csv("god.csv",index = False) #0.6526